#This case study investigates automatic classification of fashion items using a pretrained ResNet50 convolutional neural network. Leveraging transfer learning, the ImageNet-trained base model is adapted to the Fashion-MNIST dataset by modifying and fine-tuning the top layers for the target classes. The training process is closely monitored with epoch-wise accuracy metrics, providing detailed insights into model convergence, generalization performance, and the effectiveness of fine-tuning in improving classification accuracy.



In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import logging
import warnings
import sys
import os
import contextlib

warnings.filterwarnings("ignore")
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.get_logger().setLevel(logging.ERROR)

@contextlib.contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as fnull:
        old_stdout = sys.stdout
        sys.stdout = fnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

with suppress_stdout():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

NUM_CLASSES = 10
IMG_SIZE = 224
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.expand_dims(image, axis=-1)
    image = tf.image.grayscale_to_rgb(image)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs=base_model.input, outputs=outputs)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

class EpochAccuracyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        train_acc = logs.get("accuracy", 0) * 100
        val_acc = logs.get("val_accuracy", 0) * 100
        print(f"Epoch {epoch+1}: Accuracy = {train_acc:.2f}%, Val Accuracy = {val_acc:.2f}%")

# Train Before Fine-Tuning
EPOCHS_PRE = 5
print("\nTraining BEFORE Fine-Tuning...")
history_pre = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS_PRE,
    verbose=0,
    callbacks=[EpochAccuracyCallback()]
)

loss, accuracy = model.evaluate(test_ds, verbose=0)
print(f"\nFinal Accuracy BEFORE Fine-Tuning: {accuracy*100:.2f}%")

# Fine-Tuning
for layer in base_model.layers[-50:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Train After Fine-Tuning
EPOCHS_FINE = 10
print("\nTraining AFTER Fine-Tuning...")
history_fine = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS_FINE,
    verbose=0,
    callbacks=[EpochAccuracyCallback()]
)

loss, accuracy = model.evaluate(test_ds, verbose=0)
print(f"\nFinal Accuracy AFTER Fine-Tuning: {accuracy*100:.2f}%")


Training BEFORE Fine-Tuning...
Epoch 1: Accuracy = 47.61%, Val Accuracy = 64.26%
Epoch 2: Accuracy = 61.92%, Val Accuracy = 70.05%
Epoch 3: Accuracy = 64.79%, Val Accuracy = 68.75%
Epoch 4: Accuracy = 66.15%, Val Accuracy = 71.82%
Epoch 5: Accuracy = 67.37%, Val Accuracy = 73.07%

Final Accuracy BEFORE Fine-Tuning: 73.07%

Training AFTER Fine-Tuning...
Epoch 1: Accuracy = 72.91%, Val Accuracy = 81.60%
Epoch 2: Accuracy = 81.00%, Val Accuracy = 76.84%
Epoch 3: Accuracy = 83.59%, Val Accuracy = 73.13%
Epoch 4: Accuracy = 85.16%, Val Accuracy = 83.55%
Epoch 5: Accuracy = 86.08%, Val Accuracy = 80.33%
Epoch 6: Accuracy = 86.88%, Val Accuracy = 84.85%
Epoch 7: Accuracy = 87.38%, Val Accuracy = 80.32%
Epoch 8: Accuracy = 88.16%, Val Accuracy = 77.64%
Epoch 9: Accuracy = 88.47%, Val Accuracy = 83.61%
Epoch 10: Accuracy = 88.98%, Val Accuracy = 88.17%

Final Accuracy AFTER Fine-Tuning: 88.17%
